<a href="https://colab.research.google.com/github/MatthewYancey/GANime/blob/master/src/inference_lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lama Inference
This notebook takes a given lama model and runs inference on images

In [5]:
print('\n> Cloning the repo')
!git clone https://github.com/saic-mdal/lama.git

print('\n> Install dependencies')
!pip install -r lama/requirements.txt --quiet
!pip install wget --quiet

print('\n> Changing the dir to:')
% cd /content/lama

print('\n> Download the model')
!curl -L $(yadisk-direct https://disk.yandex.ru/d/EgqaSnLohjuzAg) -o lama-models.zip
!unzip lama-models.zip

print('>fixing opencv')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet


print('\n> Init mask-drawing code')
import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil

print('Downloading test data')
!curl -L $(yadisk-direct https://disk.yandex.ru/d/xKQJZeVRk5vLlQ) -o LaMa_test_images.zip
!unzip LaMa_test_images.zip


> Cloning the repo
Cloning into 'lama'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 283 (delta 73), reused 264 (delta 66), pack-reused 0
Receiving objects: 100% (283/283), 6.49 MiB | 38.66 MiB/s, done.
Resolving deltas: 100% (73/73), done.

> Install dependencies

> Changing the dir to:
/content/lama

> Download the model
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 5784M    0 5784M    0     0   9.9M      0 --:--:--  0:09:40 --:--:-- 9868k
Archive:  lama-models.zip
  inflating: LaMa_models/big-lama/config.yaml  
  inflating: LaMa_models/big-lama/models/best.ckpt  
  inflating: LaMa_models/lama-celeba-hq-pl-abl/lama-no-pl/config.yaml  
  inflating: LaMa_models/lama-celeba-hq-pl-abl/la

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
# MODEL_PATH = '/content/gdrive/MyDrive/repos/GANime/data_out/lama_model/matt_2022-02-14_00-21-53_train_lama-fourier_/models/last.ckpt'
MODEL_PATH = '/content/lama/LaMa_models/lama-places/lama-fourier'
INPUT_ZIP = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/mask_sample.zip'
INPUT_DIR = '/content/input/'
INPUT_DIR_2 = '/content/input/mask_sample/'
OUTPUT_DIR = '/content/output/'
OUTPUT_ZIP = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/lama_output_sample.zip'
# $(pwd)/LaMa_models/lama-places/lama-fourier
# indir=$(pwd)/sample outdir=$(pwd)/output

shutil.unpack_archive(INPUT_ZIP, INPUT_DIR)

In [11]:
# run to do inference on samples from LaMa
print('Predicting')
# shutil.rmtree()
# os.system(f'export TORCH_HOME=$(pwd) && export PYTHONPATH=. && python3 $(pwd)bin/predict.py model.path=$(pwd)/LaMa_models/lama-places/lama-fourier indir=/content/input/mask_sample outdir=$(pwd)/output')
!export TORCH_HOME=$(pwd) && export PYTHONPATH=. && python3 bin/predict.py model.path=/content/gdrive/MyDrive/repos/GANime/data_out/lama_model/custom_lama indir=/content/input/mask_sample outdir=$(pwd)/output

Predicting
Detectron v2 is not installed
[2022-02-18 01:04:08,351][saicinpainting.utils][WARNING] - Setting signal 10 handler <function print_traceback_handler at 0x7fbcaea57440>
[2022-02-18 01:04:08,382][root][INFO] - Make training model default
[2022-02-18 01:04:08,382][saicinpainting.training.trainers.base][INFO] - BaseInpaintingTrainingModule init called
[2022-02-18 01:04:08,383][root][INFO] - Make generator ffc_resnet
[2022-02-18 01:04:08,639][saicinpainting.training.trainers.base][INFO] - Generator
FFCResNetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): FFC_BN_ACT(
      (ffc): FFC(
        (convl2l): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1), bias=False, padding_mode=reflect)
        (convl2g): Identity()
        (convg2l): Identity()
        (convg2g): Identity()
        (gate): Identity()
      )
      (bn_l): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn_g): Identity()
      (act_l): ReLU(inp

In [ ]:
shutil.unpack_archive('/content/sample.zip', '/content/lama/sample/')